#  Open University Learning Analytics Dataset Preparation

Capstone Project

Claire Connaughton (10266499)

# Import Relevant Libraries 

In [1]:
import os
import pickle
import pydotplus
import numpy as np
import pandas as pd
from scipy import stats
from functools import reduce
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn import preprocessing
from IPython.display import Image  
from sklearn.tree import export_graphviz
from six import StringIO
from pandas_profiling import ProfileReport
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# Prepare the dataset

The OULA dataset contains 7 separate csv files. The database schema is displayed below. 
Source: https://analyse.kmi.open.ac.uk/open_dataset

![](model.png)

Each csv file will be loaded and inspected once by one to get an insight into the tables.
Each csv file will be cleaned sequentially before finally being merged into the final dataset. 

# Courses 

This file contains information about contains the list of all available modules and their presentations 

In [3]:
# Load courses table

try:
    courses = pd.read_csv('courses.csv')
    print("The 'courses' table has {} samples with {} features each.".format(*courses.shape))
    print("Percentage of duplicated values in Courses is ", courses.duplicated().sum() * 100 / len(courses))
    display(courses.info())
    print("Percentage of missing values in Courses ", courses.isnull().sum() * 100 / len(courses))
    display(courses.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'courses' table has 22 samples with 3 features each.
Percentage of duplicated values in Courses is  0.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes


None

Percentage of missing values in Courses  code_module                   0.0
code_presentation             0.0
module_presentation_length    0.0
dtype: float64


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


                Feature Description for Courses

code_module – code name of the module, which serves as the identifier.

code_presentation – code name of the presentation. It consists of the year and “B” for the presentation starting in February and “J” for the presentation starting in October.

length - length of the module-presentation in days.

The structure of B and J presentations may differ and therefore it is good practice to analyse the B and J presentations separately. Nevertheless, for some presentations the corresponding previous B/J presentation do not exist and therefore the J presentation must be used to inform the B presentation or vice versa. In the dataset this is the case of CCC, EEE and GGG modules.

*****************************************

# Assessments

This file contains information about assessments in module-presentations.
Usually, every presentation has a number of assessments followed by the final exam. 

In [4]:
# Load assessments table

try:
    assessments = pd.read_csv('assessments.csv')
    print("The 'assessments' table has {} samples with {} features each.".format(*assessments.shape))
    print("Percentage of duplicated values in Assessments is ", assessments.duplicated().sum() * 100 / len(assessments))
    display(assessments.info())
    print("Percentage of missing values in Assessments ", assessments.isnull().sum() * 100 / len(assessments))
    display(assessments.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'assessments' table has 206 samples with 6 features each.
Percentage of duplicated values in Assessments is  0.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               206 non-null    object 
 5   weight             206 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 9.8+ KB


None

Percentage of missing values in Assessments  code_module          0.0
code_presentation    0.0
id_assessment        0.0
assessment_type      0.0
date                 0.0
weight               0.0
dtype: float64


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0
1,AAA,2013J,1753,TMA,54,20.0
2,AAA,2013J,1754,TMA,117,20.0
3,AAA,2013J,1755,TMA,166,20.0
4,AAA,2013J,1756,TMA,215,30.0


                Feature Description for Assessments:

code_module – identification code of the module, to which the assessment belongs.

code_presentation - identification code of the presentation, to which the assessment belongs.

id_assessment – identification number of the assessment.

assessment_type – type of assessment. Three types of assessments exist: Tutor Marked Assessment (TMA), Computer Marked Assessment (CMA) and Final Exam (Exam).

date – information about the final submission date of the assessment calculated as the number of days since the start of the module-presentation. The starting date of the presentation has number 0 (zero).

weight - weight of the assessment in %. Typically, Exams are treated separately and have the weight 100%; the sum of all other assessments is 100%.

If the information about the final exam date is missing, it is at the end of the last presentation week.

*****************************************

Data Cleaning for Assessments is required.

In [5]:
# Change Assessments IDs from integers to categorical datatypes

assessments['id_assessment'] = assessments['id_assessment'].astype(object)

In [6]:
# Change the data type of 'date' from object to int

assessments['date'] = pd.to_numeric(assessments['date'], errors='coerce')

In [7]:
print(assessments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    object 
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), object(4)
memory usage: 9.8+ KB
None


In [8]:
# Check weightings of assessment results.
# The weighting of exams is 100%
# The weighting of the sum of assessments is 100%
# Modules with assessments and exams would have a weighting of 200%

# Determine the weightings of each module

assessments\
.groupby(['code_module','code_presentation', 'assessment_type'])\
.agg(weight_by_type = ('weight', sum))

weight_by_type
code_module code_presentation assessment_type                
AAA         2013J             Exam                      100.0
                              TMA                       100.0
            2014J             Exam                      100.0
                              TMA                       100.0
BBB         2013B             CMA                         5.0
                              Exam                      100.0
                              TMA                        95.0
            2013J             CMA                         5.0
                              Exam                      100.0
                              TMA                        95.0
            2014B             CMA                         5.0
                              Exam                      100.0
                              TMA                        95.0
            2014J             Exam                      100.0
                              TMA                       100.0
CCC         2014B             CMA                        25.0
                              Exam                      200.0
                              TMA                        75.0
            2014J             CMA                        25.0
                              Exam                      200.0
                              TMA                        75.0
DDD         2013B             CMA                        25.0
                              Exam                      100.0
                              TMA                        75.0
            2013J             Exam                      100.0
                              TMA                       100.0
            2014B             Exam                      100.0
                              TMA                       100.0
            2014J             Exam                      100.0
                              TMA                       100.0
EEE         2013J             Exam                      100.0
                              TMA                       100.0
            2014B             Exam                      100.0
                              TMA                       100.0
            2014J             Exam                      100.0
                              TMA                       100.0
FFF         2013B             CMA                         0.0
                              Exam                      100.0
                              TMA                       100.0
            2013J             CMA                         0.0
                              Exam                      100.0
                              TMA                       100.0
            2014B             CMA                         0.0
                              Exam                      100.0
                              TMA                       100.0
            2014J             CMA                         0.0
                              Exam                      100.0
                              TMA                       100.0
GGG         2013J             CMA                         0.0
                              Exam                      100.0
                              TMA                         0.0
            2014B             CMA                         0.0
                              Exam                      100.0
                              TMA                         0.0
            2014J             CMA                         0.0
                              Exam                      100.0
                              TMA                         0.0

This indicates that the modules have both assessments (100%) and exam (100%) which is why their weighting is 200.

The exeptions are:
    
    Module CCC which has a score of 200 for exams. This suggests 2 exams.
    Module GGG which has a score of 0 for assignments. This suggests no assignments.

In [9]:
# Check that there are 2 exams in Module CCC
assessments[(assessments['code_module'] == 'CCC') & (assessments['assessment_type'] == 'Exam')][['code_module', 'code_presentation', 'assessment_type']]\
.groupby(['code_module', 'code_presentation'])\
.count()

assessment_type
code_module code_presentation                 
CCC         2014B                            2
            2014J                            2

This confirms that there are two exams in Module CCC.

In [10]:
# Check that there is only 1 exam in Module GGG
assessments[(assessments['code_module'] == 'GGG') & (assessments['assessment_type'] == 'Exam')][['code_module', 'code_presentation', 'assessment_type']]\
.groupby(['code_module', 'code_presentation'])\
.count()

assessment_type
code_module code_presentation                 
GGG         2013J                            1
            2014B                            1
            2014J                            1

This confirms that there is only one exam in Module GGG.

******************************************

# Student Results (studentAssssments table)


This file contains the results of students’ assessments. 
If the student does not submit the assessment, no result is recorded. 
The final exam submissions is missing, if the result of the assessments is not stored in the system.


In [11]:
# Load the Results table

try:
    results = pd.read_csv('studentAssessment.csv')
    print("The 'Results' table has {} samples with {} features each.".format(*results.shape))
    print("Percentage of missing values in Results ", results.isnull().sum() * 100 / len(results))
    display(results.info())
    print("Percentage of duplicated values in Results ", results.duplicated().sum() * 100 / len(results))
    display(results.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")


The 'Results' table has 173912 samples with 5 features each.
Percentage of missing values in Results  id_assessment     0.0
id_student        0.0
date_submitted    0.0
is_banked         0.0
score             0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id_assessment   173912 non-null  int64 
 1   id_student      173912 non-null  int64 
 2   date_submitted  173912 non-null  int64 
 3   is_banked       173912 non-null  int64 
 4   score           173912 non-null  object
dtypes: int64(4), object(1)
memory usage: 6.6+ MB


None

Percentage of duplicated values in Results  0.0


,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78
1,1752,28400,22,0,70
2,1752,31604,17,0,72
3,1752,32885,26,0,69
4,1752,38053,19,0,79


                    Feature Description

id_assessment – the identification number of the assessment.

id_student – a unique identification number for the student.

date_submitted – the date of student submission, measured as the number of days since the start of the module presentation.

is_banked – a status flag indicating that the assessment result has been transferred from a previous presentation.

score – the student’s score in this assessment. The range is from 0 to 100. The score lower than 40 is interpreted as Fail. The marks are in the range from 0 to 100.

In [12]:
# Change the data types of id_assessment and id_student from integer to categorical

results['id_assessment'] = results['id_assessment'].astype(object)
results['id_student'] = results['id_student'].astype(object)

In [13]:
# Change the data type of 'score' from object to int

results['score'] = pd.to_numeric(results['score'], errors='coerce')

In [14]:
print((results.info()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  object 
 1   id_student      173912 non-null  object 
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 6.6+ MB
None


In [15]:
# Check whether the Assessments information is in the Results Table

def compareCols(df1, df2):

    # Show shared columns between dataframes
    # (a) Make lists of columns for each data frame
    df1Columns = df1.columns.values.tolist()
    df2Columns = df2.columns.values.tolist()

    # (b) Find column names that are the same
    diffDict = set(df1Columns) & set(df2Columns)
    
    print('Shared columns : ', diffDict, '\n')

    # (c) Make a list of the dictinary
    diffList = list(diffDict)
    # (d) Check that if values in
    # every shared column match in
    # the two dataframes
    for col in diffList:
        x = df1[col].isin(df2[col]).value_counts()
        print('Check if values are present in both dataframes:')
        print(x, '\n')

compareCols(assessments, results)

Shared columns :  {'id_assessment'} 

Check if values are present in both dataframes:
True     188
False     18
Name: id_assessment, dtype: int64 



In [16]:
# Determine what assignments are missing from the results table 

def printDiffValues(df1, df2, col):
    '''
    Show all df1.col values not present in df2.col
    '''
    # Pull out all unique values id_assessments
    df1_IDs = df1[col].unique()
    df2_IDs = df2[col].unique()

    # Compare the two lists
    # (a) Find what values are different
    diff = set(df1_IDs).difference(set(df2_IDs))
    
    # Show information for all df1.col values not presentin df2.col
    # (a) Make a list of missing values
    missingList = list(diff)
    # (b) Find these IDs in df2
    missingDf = df1[df1[col].isin(missingList)]

    return missingDf

printDiffValues(assessments, results, 'id_assessment')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0
113,EEE,2013J,30713,Exam,235.0,100.0
118,EEE,2014B,30718,Exam,228.0,100.0


All assignments missing from the Results table are exams with 100% module weight.

# Materials (VLE table)

The csv file contains information about the available materials in the VLE. 
Typically these are html pages, pdf files, etc. 
Students have access to these materials online and their interactions with the materials are recorded. 

In [17]:
# Load vle table

try:
    materials = pd.read_csv('vle.csv')
    print("The 'Materials' table has {} samples with {} features each.".format(*materials.shape))
    print("Percentage of missing values in Materials  ", materials .isnull().sum() * 100 / len(materials))
    display(materials .info())
    print("Percentage of duplicated values in Materials  ", materials.duplicated().sum() * 100 / len(materials))
    display(materials.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'Materials' table has 6364 samples with 6 features each.
Percentage of missing values in Materials   id_site              0.0
code_module          0.0
code_presentation    0.0
activity_type        0.0
week_from            0.0
week_to              0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_site            6364 non-null   int64 
 1   code_module        6364 non-null   object
 2   code_presentation  6364 non-null   object
 3   activity_type      6364 non-null   object
 4   week_from          6364 non-null   object
 5   week_to            6364 non-null   object
dtypes: int64(1), object(5)
memory usage: 298.4+ KB


None

Percentage of duplicated values in Materials   0.0


,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,?,?
1,546712,AAA,2013J,oucontent,?,?
2,546998,AAA,2013J,resource,?,?
3,546888,AAA,2013J,url,?,?
4,547035,AAA,2013J,resource,?,?


            Feature Description:
    
id_site – an identification number of the material.

code_module – an identification code for module.

code_presentation - the identification code of presentation.

activity_type – the role associated with the module material.

week_from – the week from which the material is planned to be used.

week_to – week until which the material is planned to be used.

In [18]:
# Change id_site from integer to categorical 
materials['id_site'] = materials['id_site'].astype(object)

# StudentInfo Table

This file contains demographic information about the students together with their results.

In [19]:
# Load studentInfo table

try:
    studentInfo = pd.read_csv('studentInfo.csv')
    print("The 'studentInfo' table has {} samples with {} features each.".format(*studentInfo.shape))
    print("Percentage of missing values in studentInfo  ", studentInfo.isnull().sum() * 100 / len(studentInfo))
    display(studentInfo.info())
    print("Percentage of duplicated values in studentInfo  ", studentInfo.duplicated().sum() * 100 / len(studentInfo))
    display(studentInfo.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'studentInfo' table has 32593 samples with 12 features each.
Percentage of missing values in studentInfo   code_module             0.0
code_presentation       0.0
id_student              0.0
gender                  0.0
region                  0.0
highest_education       0.0
imd_band                0.0
age_band                0.0
num_of_prev_attempts    0.0
studied_credits         0.0
disability              0.0
final_result            0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              32

None

Percentage of duplicated values in studentInfo   0.0


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


                        Feature Description

code_module – an identification code for a module on which the student is registered.

code_presentation - the identification code of the presentation during which the student is registered on the module.

id_student – a unique identification number for the student.

gender – the student’s gender.

region – identifies the geographic region, where the student lived while taking the module-presentation.

highest_education – highest student education level on entry to the module presentation.

imd_band – specifies the Index of Multiple Depravation band of the place where the student lived during the module-presentation.

age_band – band of the student’s age.

num_of_prev_attempts – the number times the student has attempted this module.

studied_credits – the total number of credits for the modules the student is currently studying.

disability – indicates whether the student has declared a disability.

final_result – student’s final result in the module-presentation.

In [20]:
# Change the data type for id_student from integer to categorical

studentInfo['id_student'] = studentInfo['id_student'].astype(object)

No further cleaning is required

************************************

# StudentRegistration table

This file contains information about the time when the student registered for the module presentation. 
For students who unregistered the date of unregistration is also recorded.  

In [21]:
# Load studentRegistration table
try:
    studentRegistration = pd.read_csv('studentRegistration.csv')
    print("The 'studentRegistration' table has {} samples with {} features each.".format(*studentRegistration.shape))
    print("Percentage of missing values in studentRegistration  ", studentRegistration.isnull().sum() * 100 / len(studentRegistration))
    display(studentRegistration.info())
    print("Percentage of duplicated values in studentRegistration  ", studentRegistration.duplicated().sum() * 100 / len(studentRegistration))
    display(studentRegistration.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'studentRegistration' table has 32593 samples with 5 features each.
Percentage of missing values in studentRegistration   code_module             0.000000
code_presentation       0.000000
id_student              0.000000
date_registration       0.138066
date_unregistration    69.097659
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   code_module          32593 non-null  object 
 1   code_presentation    32593 non-null  object 
 2   id_student           32593 non-null  int64  
 3   date_registration    32548 non-null  float64
 4   date_unregistration  10072 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB


None

Percentage of duplicated values in studentRegistration   0.0


,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


                Feature Description

code_module – an identification code for a module.

code_presentation - the identification code of the presentation.

id_student – a unique identification number for the student.

date_registration – the date of student’s registration on the module presentation, this is the number of days measured relative to the start of the module-presentation (e.g. the negative value -30 means that the student registered to module presentation 30 days before it started).

date_unregistration – date of student unregistration from the module presentation, this is the number of days measured relative to the start of the module-presentation. Students, who completed the course have this field empty. Students who unregistered have Withdrawal as the value of the final_result column in the studentInfo.csv file.

In [22]:
# Change the id_student from integer to categorical

studentRegistration['id_student'] = studentRegistration['id_student'].astype(object)

In [23]:
# Check if all student IDs recorded in the Registration tables are recorded in the Results table

compareCols(studentRegistration, results)

Shared columns :  {'id_student'} 

Check if values are present in both dataframes:
True     26746
False     5847
Name: id_student, dtype: int64 



There are 5847 students missing from the Results table.

In [24]:
# Check if there any students from the Student Information table missing from the Results table

compareCols(studentInfo, results)

Shared columns :  {'id_student'} 

Check if values are present in both dataframes:
True     26746
False     5847
Name: id_student, dtype: int64 



There 5847 students recorded in the Students Information table missing from the Assessment Results table. Are they the same students?

In [25]:
# Pull out all unique values id_assessments
df1_IDs = studentRegistration['id_student'].unique()
df2_IDs = studentInfo['id_student'].unique()

# Compare the two lists
# (a) Find what assessment IDs are different
diff = set(df1_IDs).difference(set(df2_IDs))
# (b) Count how many are different
numberDiff = len(diff)

numberDiff

0

This confirms that they are the same students.

In [26]:
# Check to see their outcome

info_not_in_results = printDiffValues(studentInfo, results, 'id_student')

column = info_not_in_results['final_result']

unique, counts = np.unique(column, return_counts = True)

dict(zip(unique, counts))

{'Fail': 1197, 'Pass': 2, 'Withdrawn': 4648}

Strangely, 2 students with no submissions recorded have passed their modules. Further investigation is required.

In [27]:
# Investigate whether there is a clerical error.
# If unregistration dates for these students are found, it is a clerical error.

info_not_in_results[info_not_in_results['final_result'] == 'Pass']

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
645,AAA,2014J,1336190,M,London Region,HE Qualification,80-90%,55<=,0,60,N,Pass
12893,CCC,2014J,1777834,F,East Anglian Region,A Level or Equivalent,80-90%,35-55,0,60,N,Pass


In [28]:
# Find unregistered date for id_student 1336190

reg_not_in_results = printDiffValues(studentRegistration, results, 'id_student')
reg_not_in_results[reg_not_in_results['id_student'] == 1336190]


,code_module,code_presentation,id_student,date_registration,date_unregistration
645,AAA,2014J,1336190,-23.0,NaN


In [29]:
# Find unregistered date for id_student 1777834

reg_not_in_results[reg_not_in_results['id_student'] == 1777834]

,code_module,code_presentation,id_student,date_registration,date_unregistration
12893,CCC,2014J,1777834,NaN,NaN


There are no unregistration dates for these 2 students indicating that it is not a clerical error

# VLE Interactions (studentVle table)

The studentVle.csv file contains information about each student’s interactions with the materials in the VLE. 

In [30]:
# Load vle_interaction table
try:
    vle_interaction = pd.read_csv('studentVle.csv')
    print("The 'vle_interaction' table has {} samples with {} features each.".format(*vle_interaction.shape))
    print("Percentage of missing values in vle_interaction  ", vle_interaction.isnull().sum() * 100 / len(vle_interaction))
    display(vle_interaction.info())
    print("Percentage of duplicated values in vle_interaction  ", vle_interaction.duplicated().sum() * 100 / len(vle_interaction))
    display(vle_interaction.head())
except:
    print("Dataset could not be loaded. Is the dataset missing?")

The 'vle_interaction' table has 10655280 samples with 6 features each.
Percentage of missing values in vle_interaction   code_module          0.0
code_presentation    0.0
id_student           0.0
id_site              0.0
date                 0.0
sum_click            0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 487.8+ MB


None

Percentage of duplicated values in vle_interaction   7.387605018357096


,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


                Feature Description

code_module – an identification code for a module.

code_presentation - the identification code of the module presentation.

id_student – a unique identification number for the student.

id_site - an identification number for the VLE material.

date – the date of student’s interaction with the material measured as the number of days since the start of the module-presentation.

sum_click – the number of times a student interacts with the material in that day.


*********************
There are 7.40% duplicate values in the studentVle file.

However, duplication is entirely acceptable here as the system most likely records the clicks at different points on the same day, leading to duplicates.

Therefore the duplicate values will remain unchanged.

*******************************

# CREATING THE FINAL DATASET

# Merging Tables Together

Merge the Materials and the VLE Interaction Tables together using an inner merge as resources with no activity for any student provide zero information. The date column will be dropped as it won't provide any extra information after grouping by module presentation per student.

In [31]:
# Merge with an inner join

VLE_Materials = pd.merge(vle_interaction, materials, on=['code_module', 'code_presentation', 'id_site'], how='inner')

# Drop columns
VLE_Materials.drop(columns=['week_from', 'week_to', 'date'], inplace=True)


 Merge the studentRegistration table with the Courses table using an inner join into regCourses. 

In [32]:
# Merge with an inner join
regCourses = pd.merge(studentRegistration , courses, on=['code_module', 'code_presentation'], how='inner')


Merge regCourses with the studentInfo table using an inner join.

In [33]:
# Merge with an inner join
regCoursesInfo = pd.merge(regCourses, studentInfo, on=['code_module', 'code_presentation', 'id_student'], how='inner')

Merge assessments and results tables. 

In [34]:
# merge with an inner join
assResults = pd.merge(assessments, results, on=['id_assessment'], how='inner')
# Rearrange column names
assResults = assResults[['id_student', 'code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'date_submitted', 'weight', 'is_banked', 'score']]

# Creating New Columns

Get the total clicks per student per module presentation

In [35]:
total_click_per_student = VLE_Materials\
.groupby(['code_module', 'code_presentation', 'id_student'])\
.agg(total_click = ("sum_click",sum))\
.reset_index()

total_click_per_student.head(7)

,code_module,code_presentation,id_student,total_click
0,AAA,2013J,11391,934
1,AAA,2013J,28400,1435
2,AAA,2013J,30268,281
3,AAA,2013J,31604,2158
4,AAA,2013J,32885,1034
5,AAA,2013J,38053,2445
6,AAA,2013J,45462,1492


Create a Weighted Score so that the total weight of all modules can be created. 

In [36]:
# Make a copy of dataset
scores = assResults

# Count how many exams there are in Results for every module presentation
scores[scores['assessment_type'] == 'Exam'][['code_module', 'code_presentation', 'id_assessment']]\
.groupby(['code_module', 'code_presentation'])\
.nunique()

id_assessment
code_module code_presentation               
CCC         2014B                          1
            2014J                          1
DDD         2013B                          1
            2013J                          1
            2014B                          1
            2014J                          1

CCC module only has results for 1 exam when the module should have 2 exams in total.

DDD module has results for the final exam (DDD module should have one exam in total).

In [37]:
### Make helper columns ###
# (a) Add column multiplying weight and score
scores['weight*score'] = scores['weight']*scores['score']
# (b) Aggregate recorded weight*score per student
    # per module presentation
sum_scores = scores\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(weightByScore = ('weight*score', sum))\
.reset_index()
# (c) Calculate total recorded weight of module
# (c.i) Get total weight of modules
total_weight = assessments\
.groupby(['code_module', 'code_presentation'])\
.agg(total_weight = ('weight', sum))\
.reset_index()
# (c.ii) Subtract 100 to account for missing exams
total_weight['total_weight'] = total_weight['total_weight']-100
# (c.iii) Mark module DDD as having 200 credits 
total_weight.loc[(total_weight.code_module == 'DDD'), 'total_weight'] = 200

### Calculate weighted score ###
# (a) Merge sum_scores and total_weight tables
score_weights = pd.merge(sum_scores, total_weight, on=['code_module', 'code_presentation'], how='inner')
# (b) Calculate weighted score
score_weights['weighted_score'] = score_weights['weightByScore'] / score_weights['total_weight']
# (c) Drop helper columns
score_weights.drop(columns=['weightByScore', 'total_weight'], inplace=True)

In [38]:
score_weights.head(7)

,id_student,code_module,code_presentation,weighted_score
0,6516,AAA,2014J,63.5
1,24734,AAA,2014J,47.5
2,26192,AAA,2014J,86.3
3,28061,AAA,2014J,77.4
4,31600,AAA,2014J,55.5
5,46844,AAA,2014J,66.5
6,52765,AAA,2014J,61.5


Create a late_rate_per_student to indicate what percentage of assignments were submitted late

In [39]:
# Calculate the difference between the submission dates
lateSubmission = assResults.assign(submission_days=assResults['date_submitted']-assResults['date'])
# Make a column indicating if the submission was late or not 
lateSubmission = lateSubmission.assign(late_submission=lateSubmission['submission_days'] > 0)

# Aggregate per student per module presentation
total_late_per_student = lateSubmission\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(total_late_submission = ('late_submission', sum))\
.reset_index()

# Make a df with total number of all assessments per student per module presentation
total_count_assessments = lateSubmission[['id_student', 'code_module', 'code_presentation', 'id_assessment']]\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.size()\
.reset_index(name='total_assessments')

# Merge df with total late assessements and total count assessments
late_rate_per_student = pd.merge(total_late_per_student, total_count_assessments, on=['id_student', 'code_module', 'code_presentation'], how='inner')
# Make a new column with late submission rate
late_rate_per_student['late_rate'] = late_rate_per_student['total_late_submission'] / late_rate_per_student['total_assessments']
# Drop helper columns
late_rate_per_student.drop(columns=['total_late_submission', 'total_assessments'], inplace=True)

late_rate_per_student

,id_student,code_module,code_presentation,late_rate
0,6516,AAA,2014J,0.000000
1,8462,DDD,2013J,0.333333
2,8462,DDD,2014J,0.000000
3,11391,AAA,2013J,0.000000
4,23629,BBB,2013B,0.750000
...,...,...,...,...
25838,2698257,AAA,2013J,0.400000
25839,2698535,CCC,2014B,0.750000
25840,2698535,EEE,2013J,0.250000
25841,2698577,BBB,2014J,0.000000


Create a fail_rate_per_student to indicate what percentage of assignments were submitted late

In [40]:
# Define function for marking failed assignments
passRate = assResults
passRate = passRate.assign(fail=passRate['score'] < 40)

# Aggregate per student per module presentation
total_fails_per_student = passRate\
.groupby(['id_student', 'code_module', 'code_presentation'])\
.agg(total_fails = ("fail",sum))\
.reset_index()

total_fails_per_student.head()

# Merge df with total fails and total count assessments
fail_rate_per_student = pd.merge(total_fails_per_student, total_count_assessments, on=['id_student', 'code_module', 'code_presentation'], how='inner')
# Make a new column with late submission rate
fail_rate_per_student['fail_rate'] = fail_rate_per_student['total_fails'] / fail_rate_per_student['total_assessments']
# Drop helper columns
fail_rate_per_student.drop(columns=['total_fails', 'total_assessments'], inplace=True)

fail_rate_per_student

,id_student,code_module,code_presentation,fail_rate
0,6516,AAA,2014J,0.00
1,8462,DDD,2013J,0.00
2,8462,DDD,2014J,0.00
3,11391,AAA,2013J,0.00
4,23629,BBB,2013B,0.00
...,...,...,...,...
25838,2698257,AAA,2013J,0.00
25839,2698535,CCC,2014B,0.75
25840,2698535,EEE,2013J,0.25
25841,2698577,BBB,2014J,0.00


# Merge All Tables

Merge assessment table

In [41]:
assessments = pd.merge(score_weights, late_rate_per_student, on=['id_student', 'code_module', 'code_presentation'], how='inner')
assessments = pd.merge(assessments, fail_rate_per_student, on=['id_student', 'code_module', 'code_presentation'], how='inner')

assessments.head()

,id_student,code_module,code_presentation,weighted_score,late_rate,fail_rate
0,6516,AAA,2014J,63.5,0.0,0.0
1,24734,AAA,2014J,47.5,0.8,0.2
2,26192,AAA,2014J,86.3,0.0,0.0
3,28061,AAA,2014J,77.4,0.2,0.0
4,31600,AAA,2014J,55.5,0.6,0.0


In [42]:
merged = pd.merge(regCoursesInfo, total_click_per_student, on=['id_student', 'code_module', 'code_presentation'], how='left')

In [43]:
merged = pd.merge(merged, assessments, on=['id_student', 'code_module', 'code_presentation'], how='left')

# Final Dataset Cleaning

In [44]:
data= merged

In [45]:
 print("The final dataset has {} samples with {} features each.".format(*data.shape))

The final dataset has 32593 samples with 19 features each.


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   code_module                 32593 non-null  object 
 1   code_presentation           32593 non-null  object 
 2   id_student                  32593 non-null  object 
 3   date_registration           32548 non-null  float64
 4   date_unregistration         10072 non-null  float64
 5   module_presentation_length  32593 non-null  int64  
 6   gender                      32593 non-null  object 
 7   region                      32593 non-null  object 
 8   highest_education           32593 non-null  object 
 9   imd_band                    32593 non-null  object 
 10  age_band                    32593 non-null  object 
 11  num_of_prev_attempts        32593 non-null  int64  
 12  studied_credits             32593 non-null  int64  
 13  disability                  325

There are missing values in the following columns:
    date_registration,
    date_unregistration,
    total_click,
    weighted_score,
    total_late_submission, 
    total_fails
    

*******************************

Treat the columns with missing values

In [47]:
# Treat the Total_click column by replacing null values with 0 
# because those students did not interact with the VLE.

data['total_click'] = data['total_click'].replace(np.nan).fillna(0)


In [48]:
# Treat weighted_score column by replacing null values with 0 
# because those students did not make any submissions

data['weighted_score'] = data['weighted_score'].replace(np.nan).fillna(0)


In [49]:
# Treat the null values in the date unregistered date_unregistered minus the median

# Make a new dataframe just with rows that have null values for the registration date
reg_date_nulls_in_reg = data\
[data['date_registration'].isnull()]



In [51]:
# Get median registration date
data.date_registration.median()

-57.0

In [57]:
# Replace date_registration NaNs with median date
data['date_registration'] = np.where( (data['date_registration'].isnull()),
                                           data.date_registration.median(),
                                           data['date_registration']
                                    )

In [56]:
# Get median unregistration date
data.date_unregistration.median()

27.0

In [58]:
# Replace date_unregistration NaNs with median date
data['date_unregistration'] = np.where( (data['date_unregistration'].isnull()),
                                           data.date_unregistration.median(),
                                           data['date_unregistration']
                                    )

In [59]:
# Treat null values in the late_rate column as 100% late 
# because they did not make any submission

data['late_rate'] = data['late_rate'].replace(np.nan).fillna(1.0)

In [60]:
# Treat null values in the fail_rate column as 100% late 
# because they did not make any submission

data['fail_rate'] = data['fail_rate'].replace(np.nan).fillna(1.0)

In [61]:
# Double check that there are no null values

data.isnull().sum()

code_module                   0
code_presentation             0
id_student                    0
date_registration             0
date_unregistration           0
module_presentation_length    0
gender                        0
region                        0
highest_education             0
imd_band                      0
age_band                      0
num_of_prev_attempts          0
studied_credits               0
disability                    0
final_result                  0
total_click                   0
weighted_score                0
late_rate                     0
fail_rate                     0
dtype: int64

All null values have been successfully treated

In [62]:
# Reset the columns so that id_student is listed first

col_list = list(data.columns)
col_list.insert(0,col_list.pop(col_list.index('id_student')))
data = data.loc[:,col_list]

In [64]:
data.head()

,id_student,code_module,code_presentation,date_registration,date_unregistration,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,total_click,weighted_score,late_rate,fail_rate
0,11391,AAA,2013J,-159.0,27.0,268,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,934.0,82.4,0.0,0.0
1,28400,AAA,2013J,-53.0,27.0,268,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1435.0,65.4,0.4,0.0
2,30268,AAA,2013J,-92.0,12.0,268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,281.0,65.4,0.4,0.0
3,31604,AAA,2013J,-52.0,27.0,268,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2158.0,76.3,0.0,0.0
4,32885,AAA,2013J,-176.0,27.0,268,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,1034.0,55.0,1.0,0.4


In [66]:
# Create new csv file containing the final dataset

data.to_csv('oulad_final.csv', index=False)